# Quantum Synth: a quantum-computer-based music synthesizer #

## Omar Costa Hamido, Mohammad Ghazi Vakili, Edoardo Giusto, Alberto Baiardi, Giovanni Amedeo Cirillo ##
### Qiskit Camp Europe, 9/13-14/2019 ###

We developed an infrastructure that permits to interface Qiskit with Max, a visual programming language for music and multimedia. Notes are encoded onto the basis states of a quantum computer, so that the musician can send commands for generating superposition of notes which must be interpreted as superposition of basis states. The results provided by the quantum computer simulator or the real device are sent back to Max, thus permitting their processing for generating sounds. Some quantum circuits have been characterized in terms of sound and a demonstration of a "phonic" interpretation of the Grover's search algorithm is also reported.


In [2]:
from qiskit import *
import numpy as np
import operator
import socket
import argparse
import random
import time
import logging
from backends_select import ChooseBackEnd
from SuperpositionGates import *
from RenormalizeProbability import *

from pythonosc import udp_client

In [3]:
# UDP_IP_SERVER="192.168.14.152"
#UDP_IP_SERVER="10.17.0.48"
#UDP_IP_SERVER="192.168.1.150"
UDP_IP_SERVER="127.0.0.1"
UDP_PORT_SERVER=7001

# global functions
log = logging.getLogger('udp_server')



def sender(results,name):
    parser = argparse.ArgumentParser()
    parser.add_argument("--ip", default=UDP_IP_SERVER,
      help="The ip of the OSC server")
    parser.add_argument("--port", type=int, default=7000,
      help="The port the OSC server is listening on")
    args = parser.parse_args()

    client = udp_client.SimpleUDPClient(args.ip, args.port)
    # for k in sorted(results):
    #     print(k, results[k])
    #     client.send_message(k, results[k])
    client.send_message(name, results)
    return True


def server(host='0.0.0.0', port=UDP_PORT_SERVER):
    s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)

    log.info("Listening on udp %s:%s" % (host, port))
    s.bind((host, port))
    return s

 ## Qiskit-Max communication for the implemented quantum gates ##
 
In the following you can find the functions permitting the communication between Qiskit and Max. It is important to precise that the result of Qiskit are provided to Max as a string of type $P_{0000} P_{0001} ... P_{1111}$ or $P_{0000} P_{0001} ... P_{1011}$ (according to the type of synthesis: additive or subtractive), where the probabilities of each basis state are separated by a space. For further details of the implemented quantum circuits, please look at the GatesNotebook Jupyter Notebook.

In [ ]:
def grover4(target, backendType, RealDeviceName,noisePresent=False,number=12):
    listForMusic = GroverSequence(target=target, initialLength=4, backendType=backendType, RealDeviceName=RealDeviceName, noisePresent=noisePresent)

    message = ""
    for k in listForMusic:
        # ii = 0
        print(k)
        if number== 12:
            k=RedistributeCell(k)
        print(k)
        for l in k:
            message = message + str(l) + " "
            # ii = ii + 1
            # if ii == 12:
            #     print(ii)
            #     break
    print(message)
    sender(results=message, name="grover")

def hadamard(backendType, RealDeviceName, noisePresent,numberShots,number=12):
    circuit=QuantumCircuit(4,4)
    Hadamard(circuit, listOfQubits=range(4))
    listForMusic = ChooseBackEnd(circuit, backendType=backendType, qubitsToBeMeasured=range(4), numberShots=numberShots, noisePresent=noisePresent, RealDeviceName=RealDeviceName,number=number)
    print(listForMusic)

    if number==12:
        listForMusic = RedistributeCell(listForMusic)

    print(listForMusic)
    sender(results=listForMusic, name="prob")

    del(circuit)

def Bell(backendType, RealDeviceName, noisePresent,numberShots,number=12):
    circuit=QuantumCircuit(4,4)
    BellStateGenerationTwoQubits(circuit)
    listForMusic = ChooseBackEnd(circuit, backendType=backendType, qubitsToBeMeasured=range(4), numberShots=numberShots, noisePresent=noisePresent, RealDeviceName=RealDeviceName)
    print(listForMusic)
    if number==12:
        listForMusic = RedistributeCell(listForMusic)
    print(listForMusic)
    sender(results=listForMusic, name="prob")
    del(circuit)

def SeperPosition(backendType, RealDeviceName, noisePresent,numberShots,number=12,notes=range(4)):
    circuit=QuantumCircuit(4,4)
    ChooseEqualSuperposition(circuit,states=notes)#we have to check togather
    listForMusic = ChooseBackEnd(circuit, backendType=backendType, qubitsToBeMeasured=range(4), numberShots=numberShots, noisePresent=noisePresent, RealDeviceName=RealDeviceName)
    print(listForMusic)
    if number==12:
        listForMusic = RedistributeCell(listForMusic)
    print(listForMusic)
    sender(results=listForMusic, name="prob")
    del(circuit)

if __name__ == '__main__':
    # app.run()
  # test=hello_world()
    FORMAT_CONS = '%(asctime)s %(name)-12s %(levelname)8s\t%(message)s'
    logging.basicConfig(level=logging.DEBUG, format=FORMAT_CONS)
    # music = QuantumCircuit(4,4)
    # ['qasm_simulator', 'ibmq_16_melbourne', 'ChooseEqualSuperposition', '1936', '0']
    s=server()

    while True:
        while True:
            (mystr, addr) = s.recvfrom(128*1024)
            print("Message from MAX:{}".format(mystr))
            mystr=str(mystr)
            mystr = mystr.split("'")
            mystr = mystr[1]
            mystr = mystr.split("stop")
            mystr = mystr[0]
            mystr = mystr.split("run")
            mystr = mystr[1]
            mystr=mystr.strip()
            mystr = mystr.split(" ")
            print(mystr)
            #targ=format(3,'#06b')[2:]
            targ=mystr[6]
            targ="0"*(4-len(targ))+targ

            print(targ)
            # listForMusic = ChooseBackEnd(music, backendType=mystr[0], qubitsToBeMeasured=range(4), numberShots=int(mystr[3]), noisePresent=True, RealDeviceName=mystr[1])
            if mystr[4]=='1':
                noise=True
            elif mystr[4]=='1':
                noise=False
            else:
                noise=False
            print(int(mystr[6],2))
            if mystr[2]=="Hadamard":
                hadamard(backendType=mystr[0], RealDeviceName=mystr[1], noisePresent=noise,numberShots=int(mystr[3]),number=int(mystr[5]))
            elif mystr[2]=="BellStateGenerationTwoQubits":
                 Bell(backendType=mystr[0], RealDeviceName=mystr[1], noisePresent=noise,numberShots=int(mystr[3]),number=int(mystr[5]))
            elif mystr[2]=="ChooseEqualSuperposition":
                SeperPosition(backendType=mystr[0], RealDeviceName=mystr[1], noisePresent=noise,numberShots=int(mystr[3]),number=int(mystr[5]))
            elif mystr[2]=="Grover":
                grover4(target=targ, backendType=mystr[0], RealDeviceName=mystr[1], noisePresent=noise,number=int(mystr[5]))
            else:
                print("Command Not defined")

2019-09-14 11:46:24,502 udp_server       INFO	Listening on udp 0.0.0.0:7001
